# Steps for building/testing MOM6 for GPU offload

This is following [the report from Marshall](https://github.com/marshallward/mom6-gpu-report/blob/main/gpu_report.rst#testing-in-mom6) which documents the compilation steps and testing. This notebook documents the process for Gadi.

In [2]:
cd /scratch/tm70/$USER
git clone --recursive https://github.com/NOAA-GFDL/MOM6-examples.git

fatal: destination path 'MOM6-examples' already exists and is not an empty directory.


: 128

In [6]:
module load intel-compiler/2021.10.0 netcdf/4.7.4 python3-as-python openmpi/4.1.7
cd MOM6-examples/ocean_only
make -j 4

bash: cd: MOM6-examples/ocean_only: No such file or directory
make -C ../shared/fms \
  BUILD=/scratch/tm70/ey7514/MOM6-examples/shared/fms/build \
  CODEBASE=/scratch/tm70/ey7514/MOM6-examples/src/FMS2
make[1]: Entering directory '/scratch/tm70/ey7514/MOM6-examples/shared/fms'
make -C /scratch/tm70/ey7514/MOM6-examples/shared/fms/build libFMS.a
make[2]: Entering directory '/scratch/tm70/ey7514/MOM6-examples/shared/fms/build'
ifort -DPACKAGE_NAME=\"FMS\" -DPACKAGE_TARNAME=\"fms\" -DPACKAGE_VERSION=\"\ \" -DPACKAGE_STRING=\"FMS\ \ \" -DPACKAGE_BUGREPORT=\"https://github.com/NOAA-GFDL/FMS/issues\" -DPACKAGE_URL=\"\" -DHAVE_SCHED_GETAFFINITY=1 -Duse_libMPI=1 -DSTDC_HEADERS=1 -DHAVE_SYS_TYPES_H=1 -DHAVE_SYS_STAT_H=1 -DHAVE_STDLIB_H=1 -DHAVE_STRING_H=1 -DHAVE_MEMORY_H=1 -DHAVE_STRINGS_H=1 -DHAVE_INTTYPES_H=1 -DHAVE_STDINT_H=1 -DHAVE_UNISTD_H=1 -DHAVE_NETCDF_H=1 -Duse_netCDF=1 -DAU_TEST_KIND_=r8_kind -DTEST_SVP_KIND_=8 -g -O2 -I/scratch/tm70/ey7514/MOM6-examples/shared/fms/build -r8   -I/scr

In [7]:
cd double_gyre
mpiexec -n 1 ../build/MOM6

NOTE: MPP_DOMAINS_SET_STACK_SIZE: stack size set to    32768.
 &MPP_IO_NML
 HEADER_BUFFER_VAL       =       16384,
 GLOBAL_FIELD_ON_ROOT_PE = T,
 IO_CLOCKS_ON    = F,
 SHUFFLE =           0,
 DEFLATE_LEVEL   =          -1,
 CF_COMPLIANCE   = F
 /
NOTE: MPP_IO_SET_STACK_SIZE: stack size set to     131072.
NOTE: ======== Model being driven by MOM_driver ========
NOTE: callTree: o Program MOM_main, MOM_driver.F90
NOTE: open_param_file: MOM_input has been opened successfully.
NOTE: open_param_file: MOM_override has been opened successfully.
 MOM_in domain decomposition
whalo =    4, ehalo =    4, shalo =    4, nhalo =    4
  X-AXIS =   44
  Y-AXIS =   40
 MOM_inc domain decomposition
whalo =    2, ehalo =    2, shalo =    2, nhalo =    2
  X-AXIS =   22
  Y-AXIS =   20
NOTE: diag_manager_mod::diag_manager_init: diag_manager is using fms2_io
NOTE: diag_manager_mod::diag_manager_init: prepend_date only supported when diag_manager_init is called with time_init present.
 MOM_in domain decompos

In [8]:
module load intel-vtune

In [15]:
rm -rf mom6-prof-vtune*
mpiexec -n 1 vtune -collect hotspots -r mom6-prof-vtune ../build/MOM6

vtune: Analyzing data in the node-wide mode. The hostname (gadi-login-08.gadi.nci.org.au) will be added to the result path/name.
vtune: Warning: Only user space will be profiled due to credentials lack. Consider changing /proc/sys/kernel/perf_event_paranoid file for enabling kernel space profiling.
vtune: Collection started.
NOTE: MPP_DOMAINS_SET_STACK_SIZE: stack size set to    32768.
 &MPP_IO_NML
 HEADER_BUFFER_VAL       =       16384,
 GLOBAL_FIELD_ON_ROOT_PE = T,
 IO_CLOCKS_ON    = F,
 SHUFFLE =           0,
 DEFLATE_LEVEL   =          -1,
 CF_COMPLIANCE   = F
 /
NOTE: MPP_IO_SET_STACK_SIZE: stack size set to     131072.
NOTE: ======== Model being driven by MOM_driver ========
NOTE: callTree: o Program MOM_main, MOM_driver.F90
NOTE: open_param_file: MOM_input has been opened successfully.
NOTE: open_param_file: MOM_override has been opened successfully.
 MOM_in domain decomposition
whalo =    4, ehalo =    4, shalo =    4, nhalo =    4
  X-AXIS =   44
  Y-AXIS =   40
 MOM_inc domai

In [20]:
vtune -report=hotspots -r mom6-prof-vtune.gadi-login-08.gadi.nci.org.au -format=csv

vtune: Using result path `/scratch/tm70/ey7514/MOM6-examples/ocean_only/double_gyre/mom6-prof-vtune.gadi-login-08.gadi.nci.org.au'
vtune: Executing actions 75 % Generating a report                              Function	CPU Time	CPU Time:Effective Time	CPU Time:Spin Time	CPU Time:Overhead Time	Module	Function (Full)	Source File	Start Address
btstep	1.331156	1.331156	0.0	0.0	MOM6	btstep	MOM_barotropic.F90	0x61af70
vertvisc_coef	0.337528	0.337528	0.0	0.0	MOM6	vertvisc_coef	MOM_vert_friction.F90	0x10af4b0
zonal_flux_adjust	0.209139	0.209139	0.0	0.0	MOM6	zonal_flux_adjust	MOM_continuity_PPM.F90	0x72d770
horizontal_viscosity	0.200004	0.200004	0.0	0.0	MOM6	horizontal_viscosity	MOM_hor_visc.F90	0xa6c430
meridional_flux_adjust	0.175979	0.175979	0.0	0.0	MOM6	meridional_flux_adjust	MOM_continuity_PPM.F90	0x71dbf0
zonal_flux_layer	0.169292	0.169292	0.0	0.0	MOM6	zonal_flux_layer	MOM_continuity_PPM.F90	0x72e5e7
vertvisc	0.119630	0.119630	0.0	0.0	MOM6	vertvisc	MOM_vert_friction.F90	0x10986c0
find_cou

In [19]:
vtune -report=hotspots -r mom6-prof-vtune.gadi-login-08.gadi.nci.org.au -format=csv -group-by=source-file-path

vtune: Using result path `/scratch/tm70/ey7514/MOM6-examples/ocean_only/double_gyre/mom6-prof-vtune.gadi-login-08.gadi.nci.org.au'
vtune: Executing actions 75 % Generating a report                              Source File Path	CPU Time	CPU Time:Effective Time	CPU Time:Spin Time	CPU Time:Overhead Time
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/core/MOM_barotropic.F90	1.516542	1.516542	0.0	0.0
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/core/MOM_continuity_PPM.F90	1.265328	1.265328	0.0	0.0
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/parameterizations/vertical/MOM_vert_friction.F90	0.702962	0.702962	0.0	0.0
[Unknown]	0.510743	0.420763	0.089980	0.0
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/parameterizations/lateral/MOM_hor_visc.F90	0.200004	0.200004	0.0	0.0
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/core/MOM_CoriolisAdv.F90	0.112330	0.112330	0.0	0.0
/scratch/tm70/ey7514/MOM6-examples/src/FMS2/diag_manager/diag_manager.F90	0.090918	0.090918	0.0	0.0
/scratch/tm70/ey75

I'm not confident that I'm using VTune correctly, but it looks like MOM_barotropic is an expensive step.

But looks like `core/MOM_barotropic.F90:btstep` is the most time-consuming function.

Turns out it's a 2.5k line subroutine: https://github.com/NOAA-GFDL/MOM6/blob/dev/gfdl/src/core/MOM_barotropic.F90#L2848

Otherwise some of the subroutines in `core/MOM_continuity_PPM.F90` like [`zonal_flux_adjust`](https://github.com/NOAA-GFDL/MOM6/blob/dev/gfdl/src/core/MOM_continuity_PPM.F90#L1094) or [`meridional_flux_adjust`](https://github.com/NOAA-GFDL/MOM6/blob/dev/gfdl/src/core/MOM_continuity_PPM.F90#L1995) look like more manageable candidates.

Let's see if the `benchmark` test case looks any different.

In [26]:
cd ../benchmark

In [28]:
rm -rf mom6-prof-vtune*
mpiexec -n 1 ../build/MOM6

NOTE: MPP_DOMAINS_SET_STACK_SIZE: stack size set to    32768.
 &MPP_IO_NML
 HEADER_BUFFER_VAL       =       16384,
 GLOBAL_FIELD_ON_ROOT_PE = T,
 IO_CLOCKS_ON    = F,
 SHUFFLE =           0,
 DEFLATE_LEVEL   =          -1,
 CF_COMPLIANCE   = F
 /
NOTE: MPP_IO_SET_STACK_SIZE: stack size set to     131072.
NOTE: MPP_DOMAINS_SET_STACK_SIZE: stack size set to   955296.
NOTE: ======== Model being driven by MOM_driver ========
NOTE: callTree: o Program MOM_main, MOM_driver.F90
NOTE: open_param_file: MOM_input has been opened successfully.
NOTE: open_param_file: MOM_override has been opened successfully.
 MOM_in domain decomposition
whalo =    4, ehalo =    4, shalo =    4, nhalo =    4
  X-AXIS =  360
  Y-AXIS =  180
 MOM_inc domain decomposition
whalo =    2, ehalo =    2, shalo =    2, nhalo =    2
  X-AXIS =  180
  Y-AXIS =   90
NOTE: diag_manager_mod::diag_manager_init: diag_manager is using fms2_io
NOTE: diag_manager_mod::diag_manager_init: prepend_date only supported when diag_manager_

: 139

Weird... Maybe it's a problem with the intel compiler? Let's try build again...

I spent an afternoong looking at this with no luck.

Mighang told me it's because of the OOM killer on the login nodes, so I needed to run the test in a job. I'm used to getting a more informative message from cgroups, but idk how it's setup on Gadi...

Anyway, looking at the results of the vtune report:

In [ ]:
# per subroutine
# generated with 
# vtune -report=hotspots -r mom6-prof-vtune.gadi-cpu-clx-1883.gadi.nci.org.au/ -format=csv > prof.csv
cd /scratch/tm70/$USER/MOM6-examples/ocean_only/benchmark
head prof.csv | column -t

Function                         CPU        Time       CPU  Time:Effective  Time         CPU                              Time:Spin                   Time      CPU  Time:Overhead  Time  Module  Function  (Full)  Source  File  Start  Address
btstep                           60.259302  60.259302  0.0  0.0             MOM6         btstep                           MOM_barotropic.F90          0x61afc0                                                                           
uct_mm_iface_progress            47.899321  47.899321  0.0  0.0             libuct.so.0  uct_mm_iface_progress            mm_iface.c                  0x18260                                                                            
uct_mm_iface_poll_fifo           40.556667  40.556667  0.0  0.0             libuct.so.0  uct_mm_iface_poll_fifo           mm_iface.c                  0x18262                                                                            
ucp_worker_progress              38.411412  38.411412  0.

In [6]:
# grouped by source
# generated with
# vtune -report=hotspots -r mom6-prof-vtune.gadi-cpu-clx-1883.gadi.nci.org.au/ -format=csv  -group-by=source-file-path > prof-grouped.csv
head prof-grouped.csv | column -t

Source                                                                                                         File        Path        CPU       Time  CPU  Time:Effective  Time  CPU  Time:Spin  Time  CPU  Time:Overhead  Time
/jobfs/126523667.gadi-pbs/0/ucx/1.17.0/build/src/uct/../../../source/ucx-1.17.0/src/uct/sm/mm/base/mm_iface.c  138.257595  138.257595  0.0       0.0                                                                        
[Unknown]                                                                                                      111.184349  107.375112  3.809237  0.0                                                                        
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/core/MOM_barotropic.F90                                        69.481020   69.481020   0.0       0.0                                                                        
/scratch/tm70/ey7514/MOM6-examples/src/MOM6/src/core/MOM_continuity_PPM.F90                                    6

Ignoring the UCX and `[Unknown]`, looks like the outcome is roughly the same where `MOM_baratropic.F90:btstep` takes the longest, followed by smaller subroutines in `MOM_continuity_PPM.F90`.